In [1]:
from src import meta, config
from src.meta import add_metaclasses_to_domain

In [2]:
add_metaclasses_to_domain(config.configuration)

### Entities In Collections

##### How can a Researcher model the world?

In [3]:
dots_research = meta.Team()
improve_product = meta.Goal()
conduct_research = meta.Responsibility()
documentation_binder = meta.Task()

documentation_binder.is_member_of(conduct_research, description='Research PriOS Improvements')
conduct_research.is_member_of(improve_product, description='Improve PriOS Product')
improve_product.is_member_of(dots_research, description='Research Contributes to Product')

In [4]:
oscar = meta.Person()
oscar.is_member_of(dots_research, description='Oscar')
oscar.is_member_of(documentation_binder, description='Working on documentation')

tim = meta.Person()
tim.is_member_of(dots_research, description='Tim')

In [5]:
meta.describe_entity(dots_research)

Team
  ('Goal', 'Research Contributes to Product')
    ('Responsibility', 'Improve PriOS Product')
      ('Task', 'Research PriOS Improvements')
        ('Person', 'Working on documentation')
  ('Person', 'Oscar')
  ('Person', 'Tim')


In [6]:
improve_product

Name: Goal
Provenance: Goal -> GrammarCollection -> object

### Measures in Collections

##### How can a Researcher generate Ontologies and Taxonomies of the world?

In [7]:
attributes = meta.Taxonomy()

synthesis = meta.Taxonomy()
synthesis_situation = meta.Attribute()
synthesis_time = meta.Attribute()
composure = meta.Attribute(name='Composure', description='Consistently keeping your cool...')

synthesis.is_member_of(attributes, description='Synthesis')
synthesis_situation.is_member_of(synthesis, description='Synthesizing the Situation')
synthesis_time.is_member_of(synthesis, description='Synthesizing through Time')
composure.is_member_of(attributes, description='Composure')

In [8]:
composure

Name: Composure
Description: Consistently keeping your cool...
Provenance: Attribute -> Quality -> Measure -> GrammarObject -> object
Implemented: False

In [9]:
meta.describe_entity(attributes)

Taxonomy
  ('Taxonomy', 'Synthesis')
    ('Attribute', 'Synthesizing the Situation')
    ('Attribute', 'Synthesizing through Time')
  ('Attribute', 'Composure')


### Assertions In Collections

In [10]:
domain_model_discussion = meta.Meeting()

oscar.asserts(
    tim, context=domain_model_discussion, measure=composure, 
    value=8.0, confidence=3.0, description='Nice job controlling yourself'
)

tim.asserts(
    oscar, context=domain_model_discussion, measure=composure, 
    value=3.0, confidence=3.0, description='Get a hold of yourself!'
)

In [11]:
oscar.describe

Counter({('Meeting', 'Attribute', 'Get a hold of yourself!'): 1})

In [12]:
tim.describe

Counter({('Meeting', 'Attribute', 'Nice job controlling yourself'): 1})

### Logic as a 1st Class Domain Object

##### Note that this logic does not exist yet

In [13]:
try: oscar.get_my_observations()
except Exception as e: print(e)

'Person' object has no attribute 'get_my_observations'


##### Lets attach new logic to a Person

In [14]:
@meta.register_method_to_context(meta.Person)
def get_my_observations(self):
    """This function provides a synthesis of assertions on an Entity"""
    key_observation_data = ['value', 'confidence', 'timestamp', 'description']
    return [
        {item: getattr(assertion, item) 
         for item in dir(assertion) 
         if not item.startswith('_') 
         and item in key_observation_data} 
        for assertion in self._members
    ]

##### Note that the logic itself is self-documenting...

In [15]:
get_my_observations

Name: get_my_observations
Description: This function provides a synthesis of assertions on an Entity
Provenance: GrammarObject -> object
Implemented: True

##### ...And obviously executable

In [16]:
oscar.get_my_observations()

[{'confidence': 3.0,
  'description': 'Get a hold of yourself!',
  'timestamp': 1553804388.407549,
  'value': 3.0}]

##### The logic is also parameterizable

In [17]:
@meta.register_method_to_context(meta.Person)
def what_is_my_name(self, default_name='No Name Given'):
    if getattr(self, 'name', None):
        return self.name
    else:
        return default_name

In [18]:
oscar.what_is_my_name()

'No Name Given'

In [19]:
oscar.what_is_my_name(default_name='John Doe')

'John Doe'

In [20]:
oscar.name = 'Oscar Rabiolo'
oscar.what_is_my_name()

'Oscar Rabiolo'